In [42]:
import numpy as np
import json
import bz2
import pandas as pd 
import os
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
import psutil
# import csv
import pickle
# import pandas as pd
from pymatgen.core import Structure
import math
pd.options.display.max_colwidth = 100
sns.set_theme(style = 'ticks')

In [ ]:
# def save_arr_to_csv(arr, path):
#     with open(path, mode = 'w', newline = '') as csvfile:
#         writer = csv.writer(csvfile)
#         writer.writerows(arr)

# def read_from_csv(path):
#     with open(path, mode = 'r') as csvfile:
#         reader = csv.reader(csvfile)
#         arr = [row for row in reader]
#     return arr

#### Custom Methods

In [43]:
def separate_forces(var_forces):
    arr = []
    for index, force_arr in enumerate(var_forces):
        if len(force_arr) > 1:
            # print(force_arr)
            # print(len(force_arr))
            for i in range(len(force_arr)):
                # print(force_arr[i])
                arr.append(force_arr[i])
        else:
            arr.append(force_arr)
    return arr

def get_coordinate(array, axis = 'x'):

    switcher = {
        "x": int(0),
        "y": int(1),
        "z": int(2),
    }

    switch = switcher.get(axis, "nothing")
    coordinate_arr = [arr[switch] for arr in array] 
    print(coordinate_arr)
    return coordinate_arr


def magnitude_vec(vector):
    print("Vector: {}".format(vector)) 
    # print(sum(pow(element, 2) for element in vector))
    # print([element for element in vector])
    print(math.sqrt(sum(np.power(element, 2) for element in vector)))
    return math.sqrt(sum(np.power(element, 2) for element in vector))

    # return math.sqrt(sum(np.power(element, 2) for element in vector))

def abs_forces_per_run(var_forces):
    arr = []
    for index, force_arr in enumerate(var_forces):
        if len(force_arr) > 1:
            print(force_arr)
            sum_arr = 0
            for i in range(len(force_arr)):
                print(force_arr[i])
                sum_arr += magnitude_vec(force_arr[i])
            arr.append(sum_arr)
        else:
            arr.append(magnitude_vec(force_arr[0]))
    return arr


def import_df(path):
    df = pd.read_pickle("{}".format(path))
    return df


def add_cum_count(df):
    cum_series = df.groupby('run').cumcount()
    cum_series = cum_series.astype(str)
    df['run'] = df['run'] + "_" + cum_series


In [44]:
def get_df(directory, overwrite = False):
    if overwrite == True:
        print("Number of files in directory: {}".format(len(os.listdir(directory))))
        files = os.listdir(directory)
        print(files)

        df_data = pd.DataFrame(columns=['run', 'energy', 'forces', 'stress_xx', 'stress_xy', 'stress_xz', 'stress_yx', 'stress_yy', 'stress_yz', 'stress_zx', 'stress_zy', 'stress_zz'])

        for file_name in files:
            print("Current file selected: {}".format(file_name))
            print('RAM memory % used:', psutil.virtual_memory()[2])
            # Getting usage of virtual_memory in GB ( 4th field)
            print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)
            with bz2.BZ2File("{}/{}".format(directory,file_name)) as file:
                for line in file:
                    line = line.decode().strip()
                    if line in {"[", "]"}:
                        continue
                    if line.endswith(" "):
                        line = line[:-1]
                    entity =json.loads(line)
                    for run in entity.keys():
                        for item in entity[run]:
                            # print(item)
                            struc = Structure.from_dict(item['structure'])
                            forces = item['forces']
                    
                            stress = item['stress']
                            energy = item['energy']
                            
                            # if energy > 0:
                            #     print("There is a positive energy entry: {}".format(energy))
                            
                            data_run = pd.DataFrame({'run': run,
                                                        "structure": [struc],
                                                        "energy": energy, 
                                                        "forces": [forces], 
                                                        'stress_xx': [stress[0][0]], 'stress_xy': [stress[0][1]], 'stress_xz':[stress[0][2]], 
                                                        'stress_yx': [stress[1][0]], 'stress_yy': [stress[1][1]], 'stress_yz': [stress[1][2]], 
                                                        'stress_zx': [stress[2][0]], 'stress_zy': [stress[2][1]], 'stress_zz': [stress[2][2]]})

                            data_run = pd.DataFrame(data_run)
                            df_data = pd.concat([df_data, data_run])

        df_data.reset_index(inplace = True, drop = True)

        add_cum_count(df_data)
        
        df_data.memory_usage()
        df_data.to_pickle("df_data.pkl")

    elif overwrite == False:
       df_data = import_df('df_data.pkl')

    return df_data

def create_arr(directory):

    print("Number of files in directory: {}".format(len(os.listdir(directory))))
    files = os.listdir(directory)
    print(files)

    dict_id = {}
    # df_id = pd.DataFrame(columns = ["id", "run"])

    stress_arr = []
    energy_arr = []
    forces_arr = []
    index = 0

    for file_name in tqdm(files):
        print("Current file selected: {}".format(file_name))
        
        with bz2.BZ2File("{}/{}".format(directory,file_name)) as file:
            for line in tqdm(file):
                line = line.decode().strip()
                if line in {"[", "]"}:
                    continue
                if line.endswith(" "):
                    line = line[:-1]
                entity =json.loads(line)
                for run in entity.keys():
                    for item in entity[run]:
                        # print(len(entity[run]))
                        struc = Structure.from_dict(item['structure'])
                        forces = item['forces']
                        # force_1 = item['forces'][0]
                        stress = item['stress']
                        energy = item['energy']
                        
                        dict_id[index] = run

                        stress_arr.append(stress)
                        forces_arr.append(forces)
                        energy_arr.append(energy)
                        
                        with open('log.txt', 'a') as f:
                            f.write("{}-{}: Energy: {}, Forces: {}, Stress: {} + \n".format(run, index, energy, forces, stress))
                        index += 1

    return energy_arr, stress_arr, forces_arr, dict_id

In [45]:
energy_arr, stress_arr, forces_arr, dict_id = create_arr("geo_opt")

Number of files in directory: 92
['2_spg29.json.bz2', '2_spg28.json.bz2', 'ml_2_A2B7.json.bz2', 'ml_2_A9B10.json.bz2', 'ml_2_A9B11.json.bz2', 'ml_3_AB2C6.json.bz2', '2_spg17.json.bz2', 'ml_2_A3B4.json.bz2', 'ml_2_A3B5.json.bz2', '1_elements.json.bz2', 'ml_2_A4B9.json.bz2', 'ml_4_ABC2D6.json.bz2', 'ext_c2db_0.25_3.json.bz2', 'ext_c2db_0.25_2.json.bz2', '2_quasi.json.bz2', '2_spg01.json.bz2', 'ml_2_A5B11.json.bz2', '2_spg34.json.bz2', '2_spg35.json.bz2', '2_spg43.json.bz2', '2_spg70.json.bz2', 'ml_3_ABC6.json.bz2', 'ml_2_AB6.json.bz2', 'ml_2_AB7.json.bz2', '2_extra01.json.bz2', '2_spg33.json.bz2', '2_spg32.json.bz2', 'ml_3_A2B2C3.json.bz2', 'ml_2_A3B11.json.bz2', 'ml_2_A3B10.json.bz2', 'ml_2_A3B8.json.bz2', 'ml_3_AB3C8.json.bz2', '3_spg33.json.bz2', 'ml_2_A4B5.json.bz2', '2_spg10.json.bz2', '2_hydrides.json.bz2', 'ml_3_ABC.json.bz2', 'ml_2_A5B7.json.bz2', 'ml_2_A5B6.json.bz2', 'ext_c2db_0.25_4.json.bz2', 'ext_c2db_0.25_5.json.bz2', 'ml_3_ABC2.json.bz2', 'ml_3_ABC3.json.bz2', '2_spg36.jso

  0%|          | 0/92 [00:00<?, ?it/s]

Current file selected: 2_spg29.json.bz2


1it [00:03,  3.65s/it]
  1%|          | 1/92 [00:03<05:32,  3.65s/it]

Current file selected: 2_spg28.json.bz2


...

1it [00:04,  4.99s/it]
 98%|█████████▊| 90/92 [1:15:21<04:39, 139.78s/it]

Current file selected: 2_spg04.json.bz2


1it [00:05,  5.31s/it]
 99%|█████████▉| 91/92 [1:15:26<01:39, 99.44s/it] 

Current file selected: 2_spg05.json.bz2


1it [00:00,  1.58it/s]
100%|██████████| 92/92 [1:15:27<00:00, 49.21s/it]


In [46]:
def save_arr_json(arr, path):
    with open(path, mode = 'w') as line:
        json.dump(arr, line)

def read_arr_json(path):
    with open(path, mode = 'r') as line:
        arr = json.load(line)
    return arr

def save_id_dict(dict, path):
    # BEWARE JSON turns keys into strings and not int
    with open(path, 'w') as line:
        json.dump(dict, line)

def load_id_dict(path):
    with open(path, 'r') as line:
        arr = json.load(line)
    return arr

In [47]:
save_arr_json(forces_arr, "forces_arr.txt")
save_arr_json(stress_arr, "stress_arr.txt")
save_arr_json(energy_arr, "energy_arr.txt")
save_id_dict(dict_id, "id_run.txt")

In [48]:
forces_arr = read_arr_json("forces_arr.txt")
energy_arr = read_arr_json("energy_arr.txt")
stress_arr = read_arr_json("stress_arr.txt")
id_run = load_id_dict("id_run.txt")

In [ ]:
def histogram_1d_energy(arr, bin = 500, xrange = [-150,100])
    fig, ax = plt.subplots()
    n, bins, patches = ax.hist(arr, bins = bin)
    ax.set_xlabel("{}".format("Energy")
    ax.set_xlim(xrange)
    ax.set_ylabel("Counts")

In [41]:
print(id_run["14000"])

runs_2/spg29/Ni/NiBr2/xxx_02p-00_29-2-Ni2a.Br4b


In [ ]:
print(np.shape(forces_arr))
print(np.shape(stress_arr))
print(stress_arr[1])
print(forces_arr[0])
print(np.shape(energy_arr))

### Import pandas dataframe
Provide the ``overwrite == True`` case to rerun the creation of the dataframe, else if ``overwrite == False`` imports it from the saved .pkl file

In [ ]:
df = get_df("geo_opt", overwrite = True)
display(df)

### Plotting Histograms

#### Energy Histogram

In [ ]:
histogram_1d(df, quantity = 'energy', bin = 200)

In [ ]:
display(df['stress_xx'])

display(df[df['stress_xx'] > 200])
histogram_1d(df, quantity = 'stress_zz', bin = 200, xrange= [-20, 20])
histogram_1d(df, quantity = 'stress_zy', bin = 200, xrange= [-20, 20])
histogram_1d(df, quantity = 'stress_zy', bin = 200, xrange= [-20, 20])

In [ ]:
force_arr = separate_forces(df['forces'].to_numpy())    
# print(force_arr)
# print(force_arr[20000])
force = get_coordinate(array= force_arr, axis = "x")
plt.hist(force, bins= 20)
# plt.hist(force, histtype='step')

#  n, bins, patches = ax.hist(force, bins = bin)
#     ax.set_xlabel("{} {}".format("Force", axis))
#     ax.set_xlim(xrange)
#     ax.set_ylabel("Counts")

In [ ]:
histogram_1d_force(df, axis = 'x', bin = 10, xrange = [0, 0.4])

In [ ]:
energy_force_combined(df)

In [ ]:
var = df["forces"].to_numpy()

fig, ax = plt.subplots()
force_arr = separate_forces(var)    

force = abs_forces_per_run(var)
print(type(force))
                            
n, bins, patches = plt.hist(force)
plt.show()

In [ ]:
abs_forces_per_run(var_forces= df["forces"].to_numpy())

In [ ]:
energy_arr, stress_arr, forces_arr = create_arr("geo_opt_test")

In [ ]:
# print(energy_arr)
# print(forces_arr)

print(len(forces_arr))
print(len(energy_arr))

In [ ]:
display(df)

In [ ]:
positive_entries = df[df['energy'] > 1000]
print(positive_entries)

In [ ]:
df = read_df(path = "df_data.pkl")

In [ ]:
display(df)
print(df.energy.max())
print(df.energy.min())

In [ ]:
histogram_1d_force(df, axis = 'x', bin = 500, xrange = [-150, 150])

In [ ]:
histogram_1d(df = df, quantity = "force_x", bin = 500, xrange = [-150,100])

In [ ]:
force_arr = separate_forces(var2)
  # print(force_arr)

  print(force_arr[1])

  x_forces = get_coordinate(array= force_arr, axis = "x")

In [ ]:
histogram_2d_energy_stress(df)

In [ ]:
histogram_1d(df = df, quantity = "energy", bin=2000)

In [ ]:
display(df['force_1'])
np.where(pd.isna(df['force_1']))

In [ ]:
path = "geo_opt/2_spg01.json.bz2"
path = "geo_opt/ml_3_AB2C4.json.bz2"
quantity = []
path_of_directory = 'geo_opt'
desired_quantity = 'energy'

# df_data = pd.DataFrame(columns = ['structure', 'force_1', 'force_2', 'stress_xx', 'stress_xy', 'stress_xz', "stress_yx", "stress_yy", 'stress_yz', 'stress_zx', 'stress_zy', 'stress_zz'])

df_data = pd.DataFrame(columns=['run', 'energy', 'force_1', 'force_2', 'stress_xx', 'stress_xy', 'stress_xz', 'stress_yx', 'stress_yy', 'stress_yz', 'stress_zx', 'stress_zy', 'stress_zz'])
# file_data = json.load(bz2.open(path, "rb"))
# print(file_data)


def retrieve_forces(force_array):
    force_1, force_2 = force_array[0], force_array[1]
    return force_1, force_2



with bz2.BZ2File(path) as file:
    for line in file:
        line = line.decode().strip()
        if line in {"[", "]"}:
            continue
        if line.endswith(" "):
            line = line[:-1]
        entity =json.loads(line)
        print(entity)
        for runs in entity.keys():
            for item in entity[runs]:
                struc = Structure.from_dict(item['structure'])
                force_1, force_2 = retrieve_forces(item['forces'])
                stress = item['stress']
    
                energy = item['energy']

                data_run = pd.DataFrame({'run': runs,
                                         "structure": [struc],
                                         "energy": energy, 
                                         "force_1": [force_1], "force_2": [force_2], 
                                         'stress_xx': [stress[0][0]], 'stress_xy': [stress[0][1]], 'stress_xz':[stress[0][2]], 
                                         'stress_yx': [stress[1][0]], 'stress_yy': [stress[1][1]], 'stress_yz': [stress[1][2]], 
                                         'stress_zx': [stress[2][0]], 'stress_zy': [stress[2][1]], 'stress_zz': [stress[2][2]]})

                data_run = pd.DataFrame(data_run)
                df_data = pd.concat([df_data, data_run])
 

df_data.memory_usage()

df_data.reset_index(inplace=True, drop =True)
cum_series = df_data.groupby('run').cumcount()
cum_series = cum_series.astype(str)
df_data['run'] = df_data['run'] + "_" + cum_series

display(df_data)
        

def plotting_function(df, desired_quantity, savename):
        counts, bins = np.histogram(df[desired_quantity])
        plt.hist(bins[:-1], bins, weights = counts)
        
        plt.savefig("Figures/{}.pdf".format(savename))


# print(dtype(df_data['energy']))

print(df_data.dtypes)

energy = df_data['energy'].to_numpy()
# print(energy)
stress_xx = df_data['stress_xx'].to_numpy()

# histo = np.histogram2d(energy, stress_xx, bins = (20, 20))
# print(type(histo))






    # fig, ax = plt.subplots()
    # hist = ax.hist2d(var1, var2, bins = (bin, bin), cmap = plt.cm.jet)
    # fig.colorbar(hist)

    # fig.tight_layout()
    # fig.savefig("Figures/{}.pdf".format(savename_fig))
    # plt.show()  








# plt.imshow(histo)

# plotting_function(df_data, "energy", "energy_dist")



#         # Get all the keys (runs)
#         # print(entity.keys())

#         for runs in entity.keys():
#             # The argument in entity specifies the run
#             for i in entity[runs]:                    
# #                 print(i.keys())
#                 value = i[desired_quantity]
#                 forces = i['forces']
#                 stress = i['stress']
#                 structure = i['structure']
#                 energy = i['energy']
#                 # print(forces)
#                 quantity.append(value)
                
# print(quantity)

In [ ]:
def return_quantity(directory, desired_quantity):
    # quantity can be structure, energy, forces, stress
#     path = "geo_opt/2_spg01.json.bz2"
    quantity = []
#     ext = (".bz2")
    print("Number of files in directory: {}".format(len(os.listdir(directory))))
    file_names = os.listdir(directory)
    # print(file_names)
#     for files in os.listdir(directory):
    for files in file_names:
        print("Current file selected: {}".format(files))
        with bz2.BZ2File("{}/{}".format(directory,files)) as file:
            for line in file:
                line = line.decode().strip()
                if line in {"[", "]"}:
                    continue
                if line.endswith(" "):
                    line = line[:-1]
                entity =json.loads(line)

                # Get all the keys (runs)
                # print(entity.keys())

                for runs in entity.keys():
                    # The argument in entity specifies the run
                    for i in entity[runs]:                    
#                         print(i.keys())
                        value = i[desired_quantity]
                        quantity.append(value)
#                         forces = i['forces']
#                         stress = i['stress']
#                         structure = i['structure']
#                         energy = i['energy']
#                         print(forces)
                
    with open("{}.txt".format(desired_quantity), 'w') as f:
        for s in quantity:
            f.write(str(s) + '\n')

    return quantity


energy = return_quantity('geo_opt','energy')

In [ ]:
with open("energy.txt", 'r') as f:
    energy = [line.rstrip('\n') for line in f]
    print(energy)

# print(energy)

In [ ]:
print(energy)

In [ ]:
counts, bins = np.histogram(energy)
plt.hist(bins[:-1], bins, weights = counts)